In [3]:
from base import base
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.tree import (
    DecisionTreeClassifier, 
    plot_tree
)
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold)
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_curve,
    auc,
    RocCurveDisplay
)


In [4]:
def split(df):
    X, y = base(df)
    X_train_cv, X_test, y_train_cv, y_test = train_test_split(X.values,
                                                          y.values,
                                                          test_size=0.2,
                                                          random_state=42,
                                                          stratify=y)
    return X_train_cv, X_test, y_train_cv, y_test

In [10]:
X_train_cv, X_test, y_train_cv, y_test = split('white')

In [11]:
def train(X, y, model_klass, model_kwargs = {}):
    cv = StratifiedKFold(n_splits=5)
    f1_score_val_list = []
    f1_score_train_list = []
    model_list =[]
    scaler_list = []
    # Validação cruzada só em Training Data
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train = X[train_idx, :]
        y_train = y[train_idx]
        X_val = X[val_idx, :]
        y_val = y[val_idx]

        # Escala
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_val_scaled = scaler.transform(X_val)

        scaler_list.append(scaler)

        # Treino
        model = model_klass(**model_kwargs)
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_train_scaled)

        y_pred_val = model.predict(X_val_scaled)
        print(f"========================= FOLD {fold} ==========================")
        print(f"Meu resultado para treino de F1-Score é {f1_score(y_train, y_pred):.2}")
        print(f"Meu resultado para validação de F1-Score é {f1_score(y_val, y_pred_val):.2}") 
        f1_score_val_list.append(f1_score(y_val, y_pred_val))
        f1_score_train_list.append(f1_score(y_train, y_pred))
        model_list.append(model)
    print()
    print()
    print(f"Meu resultado de F1-Score Médio de treino é {np.mean(f1_score_train_list): .2} +- {np.std(f1_score_train_list): .2} ")
    print(f"Meu resultado de F1-Score Médio de validação é {np.mean(f1_score_val_list): .2} +- {np.std(f1_score_val_list): .2} ")
    print()

    best_model_idx = np.argmax(f1_score_val_list)
    print(f"Meu melhor fold é: {best_model_idx} ")
    best_model = model_list[best_model_idx]

    # Fazer a inferência em Test Data
    best_scaler = scaler_list[best_model_idx]
    X_test_scaled = best_scaler.transform(X_test)
    y_pred_test = model.predict(X_test_scaled)

    print()
    print()
    print(f"Meu resultado de F1-Score para o conjunto de teste é: {f1_score(y_test, y_pred_test):.2} ")
    return best_model

In [12]:
train(X_train_cv, y_train_cv, LogisticRegression)

========================= FOLD 0 ==========================
Meu resultado para treino de F1-Score é 0.83
Meu resultado para validação de F1-Score é 0.83
========================= FOLD 1 ==========================
Meu resultado para treino de F1-Score é 0.83
Meu resultado para validação de F1-Score é 0.81
========================= FOLD 2 ==========================
Meu resultado para treino de F1-Score é 0.82
Meu resultado para validação de F1-Score é 0.84
========================= FOLD 3 ==========================
Meu resultado para treino de F1-Score é 0.83
Meu resultado para validação de F1-Score é 0.8
========================= FOLD 4 ==========================
Meu resultado para treino de F1-Score é 0.82
Meu resultado para validação de F1-Score é 0.84


Meu resultado de F1-Score Médio de treino é  0.83 +-  0.003 
Meu resultado de F1-Score Médio de validação é  0.82 +-  0.016 

Meu melhor fold é: 4 


Meu resultado de F1-Score para o conjunto de teste é: 0.82 


LogisticRegression()

In [13]:
tree_model = train(X_train_cv, y_train_cv, DecisionTreeClassifier)

========================= FOLD 0 ==========================
Meu resultado para treino de F1-Score é 1.0
Meu resultado para validação de F1-Score é 0.83
========================= FOLD 1 ==========================
Meu resultado para treino de F1-Score é 1.0
Meu resultado para validação de F1-Score é 0.82
========================= FOLD 2 ==========================
Meu resultado para treino de F1-Score é 1.0
Meu resultado para validação de F1-Score é 0.83
========================= FOLD 3 ==========================
Meu resultado para treino de F1-Score é 1.0
Meu resultado para validação de F1-Score é 0.82
========================= FOLD 4 ==========================
Meu resultado para treino de F1-Score é 1.0
Meu resultado para validação de F1-Score é 0.82


Meu resultado de F1-Score Médio de treino é  1.0 +-  0.0 
Meu resultado de F1-Score Médio de validação é  0.82 +-  0.0051 

Meu melhor fold é: 2 


Meu resultado de F1-Score para o conjunto de teste é: 0.82 
